In [1]:
import sys
import os

In [2]:
root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

In [3]:
from controlzone import Control_zone
from counter import Counter
from detector import YOLOV3Detector
from trackerspeedestimator import TrackerSpeedEstimator
from utils import *

Using TensorFlow backend.


In [4]:
video_path = os.path.join(root_dir,"videos/Road_traffic_cut.mp4")

yolov3_weights_path = os.path.join(root_dir,"yolov3_weights/pretrained-yolov3.h5")
video_capture = cv2.VideoCapture(video_path)

HEIGHT = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
WIDTH = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
FPS = int(video_capture.get(cv2.CAP_PROP_FPS))

classes = ['car', 'truck', 'motorcycle']

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output2.avi', fourcc, FPS, (WIDTH, HEIGHT))

icons = load_icons(classes)

R_col = (255, 0, 0)
G_col = (0, 255, 0)
B_col = (0, 0, 255)

counter1 = Counter(cls=classes,
                   border=(325, 400, 600, 400),
                   color=R_col,
                   draw_loc="bottom-left")

counter2 = Counter(cls=classes,
                   border=(665, 400, 912, 400),
                   color=G_col,
                   draw_loc="bottom-right")

cz1 = Control_zone(1, HEIGHT, WIDTH,
                   ((630, 342), (451, 342)), ((170, 475), (585, 475)),
                   52, col=R_col, draw_loc="top-right", speedlimit=130)

cz2 = Control_zone(2, HEIGHT, WIDTH,
                   ((675, 475), (1035, 475)), ((820, 348), (665, 348)),
                   52, col=G_col, draw_loc="top-left",speedlimit=130)

detector = YOLOV3Detector(cls=classes, weights_path=yolov3_weights_path)

trackerspeed = TrackerSpeedEstimator(video_capture=video_capture, czones=[cz1, cz2])

while video_capture.isOpened():
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    if not ret:
        break

    img = frame.copy()

    result, detections = detector.detect(img)
    if len(detections) == 0:
        continue

    trackerspeed.track(detections)

    # counting objects
    trackerspeed.map_centroid_class()

    # Couting Display
    _ = counter1.count_class(trackerspeed.objects, trackerspeed.mapped_centroid_classes)
    _ = counter2.count_class(trackerspeed.objects, trackerspeed.mapped_centroid_classes)

    counter1.count_display(img=result, icons=icons)
    counter2.count_display(img=result, icons=icons)

    cv2.namedWindow("video", cv2.WINDOW_AUTOSIZE)
    result = cz1.display_zone(img=result)
    result = cz2.display_zone(img=result)

    trackerspeed.compute_speed()
    trackerspeed.display_speed(img=result)

    trackerspeed.display_tracking(img=result)

    out.write(result)
    cv2.imshow("video", result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

detector.close()
video_capture.release()
cv2.destroyAllWindows()
out.release()